In [1]:
import pandas as pd
from fidap import fidap_client

In [2]:
fidap = fidap_client(db='sf', api_key="b8a8ebb5f4cf4983a4fa63be6ca418c2")

# Demographic Data Questions

### Which highly populated zip code had the majority of its population black americans in 2019?

In [3]:
fidap.sql("""select top 1 DIV0(black_population,total_population) as black_density, black_population, total_population, zip_code, year
from FIDAP_DATA_SOURCES.FIDAP_SCHEMA.CENSUSDATA where year='2019' order by black_population desc, black_density desc""")

,black_density,black_population,total_population,zip_code,year
0,0.859803,86706,100844,11236,2019


### Which highly populated zip code had the majority of its population white americans in 2019?

In [7]:
fidap.sql("""select top 1 DIV0(white_population,total_population) as white_density, white_population, total_population, zip_code, year
from FIDAP_DATA_SOURCES.FIDAP_SCHEMA.CENSUSDATA where year='2019' order by white_population desc, white_density desc""")

,white_density,white_population,total_population,zip_code,year
0,0.870032,97113,111620,79936,2019


### What zip code has had the highest household income by median age?

In [4]:
fidap.sql("""select top 1 median_age, median_income, zip_code from FIDAP_DATA_SOURCES.FIDAP_SCHEMA.CENSUSDATA where year='2019' and median_income is not null order by median_income desc, median_age asc""")

,median_age,median_income,zip_code
0,64.9,250000,82219


### Which zip code with a population of more than 50,000 recorded the highest population growth in any year?

In [5]:
sql_ = f"""select top 1 zip_code, 
    total_population,
    LAG(total_population) over (partition by zip_code order by year) as previous_pop,
    ZEROIFNULL(total_population - previous_pop) as diff_to_prev,
    DIV0(diff_to_prev,total_population) * 100 as percent_change,
    year 
from FIDAP_DATA_SOURCES.FIDAP_SCHEMA.CENSUSDATA where total_population > 50000
order by percent_change desc"""
fidap.sql(sql_)

,zip_code,total_population,previous_pop,diff_to_prev,percent_change,year
0,77407,60000,51142,8858,14.763333,2019
